# Sprint12課題 深層学習スクラッチ畳み込みニューラルネットワーク1

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造はsprint11で作成したFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。

ここではパディングは考えず、ストライドも1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

フォワードプロパゲーションの数式は以下のようになります。

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)にチャンネル数を限定しないConv1として実装<br>

## 【問題2】1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください
$$
N_{out} =  \frac{N_{in}+2P-F}{S} + 1\\
$$
$N_{out}$: 出力のサイズ（特徴量の数）

$N_in$ : 入力のサイズ（特徴量の数）

$P$ : ある方向へのパディングの数

$F$ : フィルタのサイズ

$S$ : ストライドのサイズ

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/utils/change_shape.py)に畳み込み後の出力サイズを求める関数をget_output_sizeとして実装

## 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力x、重みw、バイアスbを次のようにします。

In [5]:
import sys
dir_str = "../../ml-scratch/model"
if (dir_str not in sys.path):
    sys.path.append(dir_str)

import numpy as np    
from layer import Conv1D

#動作確認
x = np.array([[[1,2,3,4]]])
W = np.array([[[3, 5, 7]]])
b = np.array([1])

delta_a = np.array([[[10, 20]]])

In [6]:
conv1 = Conv1D(W=W, b=b, stride=1, pad=0)
out = conv1.forward(x)
print(out)

[[[35. 50.]]]


In [9]:
dx = conv1.backward(delta_a)
print(conv1.db)
print(conv1.dW)
print(dx)

[30]
[[[ 50.  80. 110.]]]
[[[ 30. 110. 170. 140.]]]


確認できた。

## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

紙やホワイトボードを使い計算グラフを書きながら考えてください。

問題1で取り組んだ通り、[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)にチャンネル数を限定しないConv1として実装<br>

## 【問題5】学習・推定
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えて学習と推定を行ってください。出力層だけは全結合層をそのまま使ってください。

チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、平滑化を行います。平滑化はNumPyのreshapeが使用できます。

numpy.reshape — NumPy v1.15 Manual

画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/scratch_cnn1d_classifier.py)のpyファイルにScratch1dCNNClassifierクラスを実装

In [27]:
#データセットの用意
from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [28]:
#data数を絞る
X_train = X_train[:30000]
y_train = y_train[:30000]
X_test = X_test[:500]
y_test = y_test[:500]

#reshape(N, 28, 28) -> (N, 1, 784)
X_train = X_train.reshape(-1, 1, X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(-1, 1, X_test.shape[1]*X_test.shape[2])

#normalize
X_train = X_train.astype(np.float)/255
X_test = X_test.astype(np.float)/255
X_train.shape
#one-hot
eye = np.eye(len(np.unique(y_train)))
y_train = eye[y_train]
y_test = eye[y_test]

#split into train, val
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) 
print(X_val.shape) 

(24000, 1, 784)
(6000, 1, 784)


In [29]:
from scratch_cnn1d_classifier import Scratch1dCNNClassifier
cnn = Scratch1dCNNClassifier(
    conv_param={'n_filters': 30, 'filter_size': 5, 'stride': 1, 'pad': 0},
    pool_param={'pool_size': 2},
    n_epochs=3,
    batch_size=1000,
    optimizer='Adam',
    optimizer_param={'lr': 0.001},
    layer_nodes = {'hidden': 100, 'output': 10},
    weight_init_std=0.01,
    verbose=True
)

cnn.fit(x_train=X_train, y_train=y_train, x_val=X_val, y_val=y_val)
pred = cnn.predict(X_test)

epoch: 0
train_acc: 0.5514583333333334, val_acc0.541
train loss: 1.7614287501493349, val_loss1.7614287501493349
epoch: 1
train_acc: 0.820875, val_acc0.8135
train loss: 0.6095958563446384, val_loss0.6095958563446384
epoch: 2
train_acc: 0.8866666666666667, val_acc0.8836666666666667
train loss: 0.3973413531025308, val_loss0.3973413531025308


In [31]:
y_actual = np.argmax(y_test, axis=1)
print("acuracy: ", (pred == y_actual).sum() / len(pred))

acuracy:  0.886


## 【問題6】（アドバンス課題）パディングの実装
畳み込み層にパディングを加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。

最も単純なパディングは全て0で埋めるゼロパディングであり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。

フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。

なお、NumPyにはパディングの関数が存在します。

numpy.pad — NumPy v1.15 Manual

問題1で取り組んだ通り、[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)にパディングを考慮したConv1として実装<br>

## 【問題7】（アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

問題1で取り組んだ通り、[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)にミニバッチ学習に対応したConv1として実装<br>
問題5でミニバッチ学習した。

## 【問題8】（アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。